# Benchmarking Models In Flux
In this file we will be benchmarking the following
   - Conv
   - DepthwiseConv
   - ConvTranspose
   - Dense
   - LSTM
   - RNN
   - Normalization Layers
    - GNU
    - Batch Norm
    - Dropout
   - CrossCor

In [ ]:
using Flux
using Images
using Statistics # To find the mean

# We will use this image as our training data as it will be the same for keras and flux
img = load("G:\\Book-Cover-Test\\img20.png")

# Flux expects the data to be in the format of (pixle_data, pixle_data, channels, samples)
# Currently the data is in the form of (channels, pixle_data, pixle_data, )
# we need to reshape it and add an extra dimention.
img = permutedims(channelview(img), (2,3,1))
img = Float32.(img)

# To add another dimention we are creating another array with the correct data size 
# and we're gonna add the image to this data array
data = Array{Float32}(undef, 500,425,3,1)
data[:,:,:,1] = img

# Some models, like the dense and the lstm model, work better if the data is flattend
# so we define another array with the correct dimentions and add the data
# flatening the data is analogous removing all the square brackets and putting everyting in a single
# array, all the pixle values just lined up in a vector.
flatdata = Array{Float32}(undef, 500*425*3,1)
flatdata[:,1] = reshape(img, 500*425*3)


# This function will take the data and model and run the data
# through the model 10 times and take the average of the times to 
# get a reliable value.
function get_avg_time(f, d)
    times = []
    for _ in 1:10
        t = @elapsed f(d)
        push!(times,t)
    end
    return mean(times)
end


# Defining and Running the model

In [38]:

Conv_model = Chain(
            Conv((10,10), 3=>100,pad=(1,1),stride=(3,3), relu)
)
get_avg_time(Conv_model, data)


0.20339787009999996

In [53]:

Maxpool_model = Chain(
            MaxPool((100,100))
)
get_avg_time(Maxpool_model,data)


0.0014615401000000001

In [52]:
Meanpool_model = Chain(
            MeanPool((100,100))
)
get_avg_time(Meanpool_model,data)

0.0015527497

In [3]:

Depthwise_model = Chain(
    DepthwiseConv((10,10), 3=>3,pad=(1,1), stride=(3,3),relu)
)
get_avg_time(Depthwise_model,data)


0.14178723059999998

In [42]:
ConvTranspose_model = Chain(
            ConvTranspose((10,10), 3=>36,pad=(1,1), stride=(3,3), relu)
)
get_avg_time(ConvTranspose_model,data)


4.8200052096

In [43]:
CrossCor_model= Chain(
            CrossCor((10,10), 3=>100,pad=(1,1), relu)
)
get_avg_time(CrossCor_model,data)


0.42018791999999994

In [46]:
BatchNorm_model = Chain(
            Conv((5,5), 3=>40,pad=(1,1), relu),
            BatchNorm(40, relu)
)
get_avg_time(BatchNorm_model,data)

0.1382069398

In [51]:

Dropout_model = Chain(
            Dense(500*425*3, 100, relu),
            Dropout(0.5),
            Dense(100, 10, relu),
)
get_avg_time(Dropout_model,flatdata)


0.05176838

In [48]:
Dense_model = Chain(
                Dense(500*425*3,20, relu)
)
get_avg_time(Dense_model,flatdata)


0.0231401701

In [49]:

LSTM_model = Chain(
    LSTM(500*425*3,3)
)
get_avg_time(LSTM_model,flatdata)


0.007619559699999999

In [50]:
RNN_model = Chain(
                RNN(500*425*3,3)
)
get_avg_time(RNN_model,flatdata)


0.0028059902000000005

In [61]:
GRU_model = Chain(
    GRU(500*425*3,3)
)
get_avg_time(GRU_model,flatdata)


0.007091450000000001